In [1]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from scipy.stats import chi2
from scipy.stats import norm
import pylab as plt

In [2]:
# Se cargan los datos
y_total = np.genfromtxt("msd_genre_dataset.txt",dtype =None,usecols=0,comments=None,delimiter=',',skip_header=10)
x_total = np.genfromtxt("msd_genre_dataset.txt",comments=None,delimiter=',',skip_header=10)[:,4:]

generos = [b'metal',b'punk',b'dance and electronica']

y_raw = []
x_raw = []
for i in range(0,y_total.size):
    if y_total[i] in generos:
        y_raw.append(y_total[i])
        x_raw.append(x_total[i])
        
y_raw = np.asarray(y_raw)
x_raw = np.asarray(x_raw)

In [3]:
# Preprocesamiento
x_normalizado = (x_raw - np.mean(x_raw,axis=0))/np.std(x_raw,axis=0)

# PCA
proporcion_varianza = 0
n_componentes = 0
while proporcion_varianza < 0.90:
    n_componentes = n_componentes + 1
    pca = PCA(n_components=n_componentes)
    pca = pca.fit(x_normalizado)
    proporcion_varianza = np.sum(pca.explained_variance_ratio_)
    
suma_acumulada_varianza = np.cumsum(pca.explained_variance_ratio_*100)

x_preprocesamiento = pca.fit_transform(x_normalizado)
xy_preprocesamiento = np.append(x_preprocesamiento,np.array([y_raw]).T,axis=1)

# Separacion de datos para prueba y entrenamiento
proporcion_prueba = 0.1
np.random.shuffle(xy_preprocesamiento)
n_prueba = round(proporcion_prueba*y_raw.size)
n_entrenamiento = y_raw.size - n_prueba
datos_prueba = xy_preprocesamiento[0:n_prueba,:]
x_prueba = datos_prueba[:,0:-1].astype(np.float64)
y_prueba = datos_prueba[:,-1]
datos_entrenamiento = xy_preprocesamiento[n_prueba:,:]
x_entrenamiento = datos_entrenamiento[:,0:-1].astype(np.float64)
y_entrenamiento = datos_entrenamiento[:,-1]

In [4]:
etiquetas_prueba = np.where(y_prueba == generos[2], np.ones(n_prueba), np.zeros(n_prueba))
etiquetas_entrenamiento = np.where(y_entrenamiento == generos[2], np.ones(n_entrenamiento), np.zeros(n_entrenamiento))

In [5]:
x_entrenamiento_logreg = np.append(np.array([np.ones(n_entrenamiento)]).T,x_entrenamiento,axis=1)
clasificador_logreg = LogisticRegression(solver='sag',max_iter=1000)
clasificador_logreg = clasificador_logreg.fit(x_entrenamiento_logreg,etiquetas_entrenamiento)

clasificador_logreg_sin_var = LogisticRegression(solver='sag',max_iter=10000)
clasificador_logreg_sin_var = clasificador_logreg_sin_var.fit(np.array([np.ones(n_entrenamiento)]).T,etiquetas_entrenamiento)

In [6]:
probabilidades_estimadas = clasificador_logreg.predict_proba(x_entrenamiento_logreg)[:,1]
probabilidades_estimadas_sin_var = clasificador_logreg_sin_var.predict_proba(np.array([np.ones(n_entrenamiento)]).T)[:,1]

In [7]:
log_verosimilitud = 0
for i in range(0,n_entrenamiento):
    log_verosimilitud = log_verosimilitud + etiquetas_entrenamiento[i]*np.log(probabilidades_estimadas[i]) + (1-etiquetas_entrenamiento[i])*np.log(1-probabilidades_estimadas[i])

In [8]:
log_verosimilitud_sin_var = 0
for i in range(0,n_entrenamiento):
    log_verosimilitud_sin_var = log_verosimilitud_sin_var + etiquetas_entrenamiento[i]*np.log(probabilidades_estimadas_sin_var[i]) + (1-etiquetas_entrenamiento[i])*np.log(1-probabilidades_estimadas_sin_var[i])

In [9]:
G = -2*(log_verosimilitud_sin_var-log_verosimilitud)

In [10]:
p_val = 1-chi2.cdf(G,x_entrenamiento.shape[1])
p_val

0.0

In [11]:
matriz_V = np.zeros([n_entrenamiento,n_entrenamiento])
for i in range(0,n_entrenamiento):
    matriz_V[i,i] = probabilidades_estimadas[i]*(1-probabilidades_estimadas[i])

In [12]:
w_logreg = clasificador_logreg.coef_
matriz_informacion = np.dot(x_entrenamiento_logreg.T,np.dot(matriz_V,x_entrenamiento_logreg))
matriz_varianza_w = np.linalg.inv(matriz_informacion)

In [13]:
estadistico_prueba = w_logreg/np.sqrt(np.diag(matriz_varianza_w ))

In [14]:
estadistico_prueba

array([[  0.7248539 ,  47.81261084, -11.21216776,  -9.8072877 ,
        -23.18880611,  -4.51699831, -20.57352632, -11.14402613,
         -0.33615705,   6.94728974,  -4.09772927,  -4.21487517,
         -5.04357664,  -2.10236908,  -3.34205709,  10.19238052,
          3.0660476 ,  -0.88221512,   1.27336193]])

In [15]:
p_values_EP = 2*(1-norm.cdf(abs(estadistico_prueba)))[0]

In [16]:
var_significativas = np.where(p_values_EP<0.05,np.ones(estadistico_prueba.size),np.zeros(estadistico_prueba.size))

In [17]:
var_significativas

array([ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  0.,  0.])

In [18]:
x_logreg_reducido = np.copy(x_entrenamiento_logreg)
for i in range(0,var_significativas.size):
    if var_significativas[var_significativas.size-i-1] == 0:
        x_logreg_reducido = np.delete(x_logreg_reducido,var_significativas.size-i-1,axis=1)

In [19]:
clasificador_logreg_red = LogisticRegression(solver='sag',max_iter=1000)
clasificador_logreg_red = clasificador_logreg_red.fit(x_logreg_reducido,etiquetas_entrenamiento)
probabilidades_estimadas_red = clasificador_logreg_red.predict_proba(x_logreg_reducido)[:,1]
log_verosimilitud_red = 0
for i in range(0,n_entrenamiento):
    log_verosimilitud_red = log_verosimilitud_red + etiquetas_entrenamiento[i]*np.log(probabilidades_estimadas_red[i]) + (1-etiquetas_entrenamiento[i])*np.log(1-probabilidades_estimadas_red[i])

In [20]:
G_red = -2*(log_verosimilitud_red-log_verosimilitud)
p_val_red = 1-chi2.cdf(G_red,x_entrenamiento.shape[1]-sum(var_significativas)+1)
p_val_red

0.62597159382288525

In [21]:
x_prueba_logreg = np.append(np.array([np.ones(n_prueba)]).T,x_prueba,axis=1)
puntaje = clasificador_logreg.score(x_prueba_logreg,etiquetas_prueba)
puntaje

0.8701171875

In [22]:
x_logreg_reducido_prueba = np.copy(x_prueba_logreg)
for i in range(0,var_significativas.size):
    if var_significativas[var_significativas.size-i-1] == 0:
        x_logreg_reducido_prueba = np.delete(x_logreg_reducido_prueba,var_significativas.size-i-1,axis=1)

In [23]:
puntaje_red = clasificador_logreg_red.score(x_logreg_reducido_prueba, etiquetas_prueba)
puntaje_red

0.8701171875